pytorch チュートリアル  
[公式doc](http://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py) 
[参考](https://qiita.com/kenchon/items/a31937cbe6385a19b82f)
- Neural Networks


In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward関数を定義するだけで，backward関数は自動的に定義できます。このbackward関数というのは，勾配が計算される関数です。これがautogradの便利なところです。
forward関数では，Tensorに関する全ての演算を使うことができます。


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
# 5 x 5が 6行

10
torch.Size([6, 1, 5, 5])


In [3]:
input = torch.randn(1, 1, 32, 32, dtype=torch.float32)
out = net(input)
print(out)

tensor([[ 0.0561, -0.0994,  0.0618, -0.0934,  0.1443, -0.0373, -0.1316,  0.0386,
          0.0377,  0.0643]], grad_fn=<AddmmBackward>)


# 損失関数
損失関数は (output, target) という２つの入力を受け取り，outputがtargetとどれだけ離れているかを計算します。

nnパッケージには，いくつかの損失関数が実装されています。シンプルな損失関数として，nn.MSELossを取り上げてみます。これは，入力とターゲットの平均二乗誤差を計算するものです。

例えば，

In [9]:
output = net(input)
target = Variable(torch.arange(1, 11, dtype=torch.float32))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target) # target = true answer
print(loss)


tensor(38.4000, grad_fn=<MseLossBackward>)


In [10]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

loss.backwardを呼び出した時，計算フロー全体は損失に関して微分される。そして，フロー（グラフ中）のVariableは.gradVariableを持つ。

例として，損失関数から前の計算ステップにさかのぼってみましょう：

# 逆伝播
誤差を逆伝播させるためには，loss.backward()するだけで良いです。
net.zero_grad()で勾配をすべて0にすることができますが，これを呼び出さなければ勾配は既存の勾配に蓄積されていきます。

それではloss.backward()を実際によびだして，その前後でconv1のバイアスの勾配がどのように変わるか見てみましょう。

In [11]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0838, -0.0203, -0.0022, -0.0385, -0.0378, -0.0380])


# パラメータの更新
最も簡単なパラメータ更新方法としては，確率的勾配降下法(SGD)が知られています。これは次の式で表現できます：
weight = weight - learning_rate * gradient

この更新規則は，次のコードで表現できます：

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

しかし，SGD以外にもNesterov-SGD, Adam, RMSPropなどのパラメータ更新法を適用したい人もいるでしょう。これを可能にするのがtorch.optimパッケージです。このパッケージにはすでにこれらの更新規則が実装されており，簡単に使うことができます。

In [12]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update


tensor([-0.0838, -0.0203, -0.0022, -0.0385, -0.0378, -0.0380])